In [1]:
import pandas as pd

In [2]:
policy_filepath = "policy_database.xlsx"
codebook_filepath = "policy_codebook.xlsx"

policies = pd.read_excel(policy_filepath)
codebook = pd.read_excel(codebook_filepath)

In [3]:
policies[:5]

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,Alabama,1991,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15
1,Alabama,1992,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
2,Alabama,1993,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
3,Alabama,1994,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
4,Alabama,1995,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16


In [4]:
codebook[:5]

,Category Code,Category,Sub-Category,Variable Name,Brief Description of Provision,Detailed Description of Provision,Coding Notes,Coding Instructions,Notes,Data Source and Attribution
0,1,Dealer regulations,Licensing,dealer,State dealer license required for sale of all ...,All firearm dealers are required to have a sta...,State requires all persons engaged in the busi...,If all firearm dealers are required to have a ...,NaN,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
1,1,Dealer regulations,Licensing,dealerh,State dealer license required for sale of hand...,All firearm dealers that sell handguns are req...,State requires all persons engaged in the busi...,If only dealers of handguns or handguns AND as...,NaN,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
2,1,Dealer regulations,Recordkeeping,recordsdealer,Licensed dealers are required to keep and reta...,Licensed dealers are required to keep and reta...,Recordkeeping refers to the recording of sales...,If record keeping and retention is required on...,Federal law requires licensed dealers to keep ...,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
3,1,Dealer regulations,Recordkeeping,recordsdealerh,Licensed dealers are required to keep and reta...,Licensed dealers are required to keep and reta...,Recordkeeping refers to the recording of sales...,If record keeping and retention is required on...,Federal law requires licensed dealers to keep ...,"Coded by Michael Siegel, MD, MPH, Boston Unive..."
4,1,Dealer regulations,Recordkeeping,recordsall,All private sellers and licensed dealers are r...,Both private sellers and licensed dealers are ...,Recordkeeping refers to the recording of sales...,If all private sellers and licensed dealers ar...,Federal law requires licensed dealers to keep ...,"Coded by Michael Siegel, MD, MPH, Boston Unive..."


## Getting policy counts for categories

In [5]:
sub_categories = {}
for category in set(codebook['Sub-Category']):
    sub_categories[category] = list(codebook[codebook['Sub-Category'] == category]['Variable Name'])

In [6]:
metadata = {'year':[], 'state':[], 'category':[], 'sub_category':[], 'policies_implemented':[]}
for i in range(len(policies)):
    for j, (sub_category, variables) in enumerate(sub_categories.items()):
        metadata['year'].append(policies.year[i])
        metadata['state'].append(policies.state[i])
        metadata['category'].append(codebook[codebook['Sub-Category'] == sub_category].Category.iloc[0])
        metadata['sub_category'].append(sub_category)
        policies_implemented = 0
        for variable in variables:
            policies_implemented += policies[variable][i]
        metadata['policies_implemented'].append(policies_implemented)

In [8]:
metadata_df = pd.DataFrame(metadata)
metadata_df.sort_values(['year', 'state', 'category'], inplace=True)
metadata_df = metadata_df[metadata_df.year > 2013]
metadata_df = metadata_df[metadata_df.year < 2021]
metadata_df

,year,state,category,sub_category,policies_implemented
1158,2014,Alabama,Ammunition regulations,Background checks,3
1187,2014,Alabama,Ammunition regulations,Prohibitors,0
1151,2014,Alabama,Assault weapons and large-capacity magazines,Assault weapons ban,0
1189,2014,Alabama,Assault weapons and large-capacity magazines,Large capacity magazine ban,0
1153,2014,Alabama,Background checks,Gun shows,0
...,...,...,...,...,...
74959,2020,Wyoming,Prohibitions for high-risk gun possession,Violent Misdemeanor,0
74962,2020,Wyoming,Prohibitions for high-risk gun possession,Felony,1
74974,2020,Wyoming,Prohibitions for high-risk gun possession,Alcohol,0
74980,2020,Wyoming,Prohibitions for high-risk gun possession,Drugs,0


In [9]:
metadata_df[metadata_df.policies_implemented > 0]

,year,state,category,sub_category,policies_implemented
1158,2014,Alabama,Ammunition regulations,Background checks,3
1178,2014,Alabama,Buyer regulations,Permitting,1
1165,2014,Alabama,Dealer regulations,Reporting,2
1181,2014,Alabama,Dealer regulations,Security,1
1182,2014,Alabama,Dealer regulations,Licensing,1
...,...,...,...,...,...
74988,2020,Wyoming,Buyer regulations,Age restrictions,2
74979,2020,Wyoming,Immunity,Immunity,1
74960,2020,Wyoming,Possession regulations,School zones,1
74995,2020,Wyoming,Possession regulations,Campus carry,2


In [10]:
metadata_df.to_csv('policy_metadata.csv', index=False)